# import packages
The below work is inspired by this link
https://plot.ly/python/big-data-analytics-with-pandas-and-sqlite/

The below work is also from:
https://www.kaggle.com/prashantrenu/explore-large-data-python-sqlite-seaborn

In [1]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout
import re

# set up directories

In [2]:
import os
os.getcwd()

'/Users/rachel/Documents/kaggle/kaggle_instacart/ipynb'

In [3]:
print("The default directory is {}".format(os.getcwd()))
path = "../" # one level above of the current directory
os.chdir(path) # convert the directory to one level above
print("The directory has now been changed to {}".format(os.getcwd()))

parent_dir = os.getcwd()
print("parent dir is always {}".format(parent_dir))
input_dir = "/raw_data" #the file is opened in a subfolder, need to go one level above and choose a different subfolder
output_dir = "/output"

The default directory is /Users/rachel/Documents/kaggle/kaggle_instacart/ipynb
The directory has now been changed to /Users/rachel/Documents/kaggle/kaggle_instacart
parent dir is always /Users/rachel/Documents/kaggle/kaggle_instacart


In [4]:
from subprocess import check_output
print("Raw_data dir contains the below files:")
print ("\n")
print(check_output(["ls", parent_dir + input_dir]).decode("utf8"))

Raw_data dir contains the below files:


aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv



### create an connection to the db

In [5]:
db_dir = parent_dir + "/db/"
print(db_dir)
conn = create_engine('sqlite:///kaggle_instakart.db')

/Users/rachel/Documents/kaggle/kaggle_instacart/db/


# import csv data into sqlite

In [6]:
csvFiles = [file for file in os.listdir(parent_dir + input_dir + "/") if file.find(".csv") != -1]
print(csvFiles)

['aisles.csv', 'departments.csv', 'order_products__prior.csv', 'order_products__train.csv', 'orders.csv', 'products.csv', 'sample_submission.csv']


In [7]:
file= 'departments.csv'
re.findall('.csv',file)
file.replace('.csv','')
#re.compile(file).sub("*.csv","")

'departments'

In [8]:
# import re
# for file in csvFiles:
#     csv = pd.read_csv(parent_dir + input_dir + "/" + file, engine = 'c')
#     tableName = file.replace('.csv','')
#     csv.to_sql(tableName, conn, if_exists = 'replace', index = False)
#     print (' Processing of Table {} ended'.format(tableName))

# investigate the tables and join them together (use SQL only)

In [9]:
"""select from all the tables in db and inspect the top 5 rows"""
dbTableNameList = [file.replace('.csv','') for file in csvFiles]
print(dbTableNameList)

tableNameDict = dict()
print(tableNameDict)

'''read the first 5 rows of each table'''
for table in dbTableNameList:
    sql_query = '''select * from ''' + table + ''';'''
    print('\n')
    print(table)
    t = pd.read_sql_query(sql_query, conn)
    tableNameDict[table] = t
    print(tableNameDict[table].head(10))

# aisles = pd.read_sql_query("SELECT * FROM aisles LIMIT 5;", conn)
# print('Total aisles: {}'.format(aisles.shape[0]))
# print("Aisles Table\n",aisles.head(),"\n\n")

['aisles', 'departments', 'order_products__prior', 'order_products__train', 'orders', 'products', 'sample_submission']
{}


aisles
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation
5         6                       other
6         7               packaged meat
7         8             bakery desserts
8         9                 pasta sauce
9        10            kitchen supplies


departments
   department_id       department
0              1           frozen
1              2            other
2              3           bakery
3              4          produce
4              5          alcohol
5              6    international
6              7        beverages
7              8             pets
8              9  dry goods pasta
9             10             bulk


order_products__prior
   order_id  prod

In [10]:
# ''' check the table individually and summarise in pandas'''

# # data = tableNameDict['orders']
# # #print(data.head())

# # data.loc[data['eval_set'] == 'test']
# # print(data.loc[data['user_id'] == 1])
# #byeval = data.groupby('eval_set')
# #print(byeval.sum())
# # pd.DataFrame(orders.groupby(['product_name']).agg({'order_id':pd.Series.nunique})
# #                          .rename(columns={'order_id':'cnt_ord_by_prod'})).reset_index()

# # data2 = tableNameDict['order_products__prior']
# # print(data2.loc[data2['order_id'] == 2774568])

# # #print(data.loc[data['order_id'] == 2539329])

# # data3 = tableNameDict['order_products__train']
# # print(data3.loc[data3['order_id'] == 2398795])


# # merge order table with all orders from users and when the order happened, since last order how long it took the user to order again
# # also order_prior shows the products ordered by the order id
# # user_id = 1

# left = data.loc[data['user_id'] == 1]
# right = data2 #prior
# right2 = data3 # train
# key = 'order_id'
# result = pd.merge(left, right, on=key)
# result2 = pd.merge(left, right2, on=key)
# print(result)
# print(result2)

# # train is the latest order history with products, prior is before train, 
# # order is a summary for all users and their historical transactions (not products)
# # pd.merge()
# # order table is the current order (latest), order prior is the previous history of the customer order



# #how many users in prior
# # print(len(data['user_id'].unique()))
# # print(data.head())


# # prior should have all customers prior order 

# print(data2.head())
# print(data3.head())
# # # print(data.head())

# # print(data.loc[data['eval_set'] == 'test'])


# # no test user prior transactions?




# # orders and prior inner join to get all train users prior history, then the train table
# # train user inner join to orders and prior
# #join order and prior first to have all users
# #trian table to select train customers
# # then the rest will be test cusotmers

# Join tables
- departments, aisles names can be joined to product table
- prior, train and possibly test are to separate the product table
- test table is used to predict next order?

In [11]:
'''Combine aisles, departments and products to create goods table'''
# start = dt.now()
drop_goods_table=""" DROP TABLE IF EXISTS goods;"""
conn.execute(drop_goods_table)

join_prod_aisle_department = """
                            create table goods as                           
                            select
                                p.*,
                                a.aisle_id as aisle_id_a,
                                a.aisle,
                                d.department_id as department_id_d,
                                d.department

                            from products p
                                left join aisles a on p.aisle_id = a.aisle_id
                                left join departments d on d.department_id = p.department_id;"""

conn.execute(join_prod_aisle_department)

goods = pd.read_sql_query("SELECT * FROM goods Limit 5;", conn)
# goods_step_time = dt.now()

In [ ]:
'''create a gigantic table for orders and prior products ordered by all customers'''
drop_orders_prods_table=""" DROP TABLE IF EXISTS prod_combined;"""
conn.execute(drop_orders_prods_table)

join_prod_order_combined = """\
                            create table prod_combined as                           
                            select
                                 p.*,
                                 prior.product_id,
                                 prior.add_to_cart_order,
                                 prior.reordered
                            from orders p
                                left join order_products__prior prior
                             on prior.order_id = p.order_id
                                    ;"""

conn.execute(join_prod_order_combined)

prod_combined = pd.read_sql_query("SELECT * FROM prod_combined Limit 5;", conn)

In [13]:
'''it is important to create index'''
index_columns = [col for col in prod_combined.columns if col.find("_id")>-1]
for col in index_columns:
    create_indexes="CREATE INDEX index_%s on prod_combined (%s);" %(col+"_prod_combined",col)
    print(create_indexes)
    conn.execute(create_indexes) 

CREATE INDEX index_order_id_prod_combined on prod_combined (order_id);
CREATE INDEX index_user_id_prod_combined on prod_combined (user_id);
CREATE INDEX index_product_id_prod_combined on prod_combined (product_id);


In [14]:
print(prod_combined.head)

<bound method NDFrame.head of    order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2539329        1    prior             1          2                  8   
2   2539329        1    prior             1          2                  8   
3   2539329        1    prior             1          2                  8   
4   2539329        1    prior             1          2                  8   

  days_since_prior_order  product_id  add_to_cart_order  reordered  
0                   None         196                  1          0  
1                   None       12427                  3          0  
2                   None       14084                  2          0  
3                   None       26088                  4          0  
4                   None       26405                  5          0  >


In [15]:
# '''use pandas to create a gigantic table for orders and prior products ordered by all customers'''
# df_orders = tableNameDict['orders']
# df_prior = tableNameDict['order_products__prior']
# df_train = tableNameDict['order_products__train']

# key = 'order_id'
# '''To get all historical order for both train and test users'''
# df_all_hist = pd.merge(df_orders, df_prior, on = key , how = 'left')
# print(df_all_hist.head(5))
# print(len(df_all_hist.user_id.unique()))

# '''To get train current orders'''
# df_train_curr = pd.merge(df_orders, df_train, on = key,how = 'inner' )
# train_users = df_train_curr['user_id'].unique()
# print(train_users)
# print(len(train_users))

In [16]:
# '''to get train user history only'''
# df_all_train_hist = df_all_hist.loc[df_all_hist['user_id'].isin(train_users)]
# print(df_all_train_hist.head())
# print(len(df_all_train_hist.user_id.unique()))

# '''to get test user history only'''
# df_all_test_hist = df_all_hist.loc[df_all_hist['user_id'].isin(train_users) == False]
# print(df_all_test_hist.head())
# print(len(df_all_test_hist.user_id.unique()))

# # do not use this one, too slow!
# # df_all_hist['train_test_flag'] = df_all_hist['user_id'].map(lambda x: 'train' if x in train_users else 'test')
# # print(df_all_hist.head)

In [17]:
'''create a table with everything'''
drop_prior_datamart_table=""" DROP TABLE IF EXISTS prior_datamart;"""
conn.execute(drop_prior_datamart_table)
join_prior_datamart_sql = """\
    CREATE TABLE prior_datamart AS
    SELECT o.*, gd.product_id, gd.product_name,gd.department,gd.aisle
    FROM prod_combined o
    INNER JOIN goods gd
    ON o.product_id = gd.product_id;
    """
conn.execute(join_prior_datamart_sql)
prior_datamart = pd.read_sql_query("SELECT * FROM prior_datamart Limit 5;", conn)

In [18]:
prior_datamart.drop('product_id:1', axis=1, inplace=True)

index_columns = [col for col in prod_combined.columns if col.find("_id")>-1]

print(index_columns)

for col in index_columns:
    create_indexes="CREATE INDEX index_%s on prior_datamart (%s);" %(col+"_prior_datamart",col)
    print(create_indexes)
    conn.execute(create_indexes)        

#print('\nTotal Time taken to delete and create prior_datamart table {}\n'.format((prior_datamart_time-orders_combined_time).total_seconds()))
print("prior_datamart Table \n", prior_datamart.head(10),"\n\n")

['order_id', 'user_id', 'product_id']
CREATE INDEX index_order_id_prior_datamart on prior_datamart (order_id);
CREATE INDEX index_user_id_prior_datamart on prior_datamart (user_id);
CREATE INDEX index_product_id_prior_datamart on prior_datamart (product_id);
prior_datamart Table 
    order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   3139998      138    prior            28          6                 11   
1   1977647      138    prior            30          6                 17   
2    389851      709    prior             2          0                 21   
3    652770      764    prior             1          3                 13   
4   1813452      764    prior             3          4                 17   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     3.0           1                  5          0   
1                    20.0           1                  1          1   
2                     6.0           1     

In [19]:
# del goods, orders_combined,prior_datamart

In [20]:
print(prior_datamart.head())
print(prior_datamart.shape)

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   3139998      138    prior            28          6                 11   
1   1977647      138    prior            30          6                 17   
2    389851      709    prior             2          0                 21   
3    652770      764    prior             1          3                 13   
4   1813452      764    prior             3          4                 17   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     3.0           1                  5          0   
1                    20.0           1                  1          1   
2                     6.0           1                 20          0   
3                     NaN           1                 10          0   
4                     9.0           1                 11          1   

                 product_name department          aisle  
0  Chocolate Sandwich Cookies     snacks  cookies ca

## Analyse the products
- the analysis will cover univariate and bivariate analysis
- plots are included
- the analysis will cover both product top line and order behaviour analysis

What else?
- how many products the super market are selling under how many departments
- where products are placed

In [21]:
import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout 

In [22]:
"""
Univariate Analysis
1. When do people order (Distribution of Time of Day) ?
2. Day of Week (Distribution of day_of_week)?
3. When do they order again (Distribution of Time Since Prior Order)?
4. How many prior orders are there (Distribution of Reorders)?

5. how often people order for similar products, say products from the same department
6. in train data, we have 13000 customers' order history, for every product they bought, then we can use that to predict what other products they reorder in the curr history the latest
7. while for the test cutomers, they also have history, maybe people who are similar shoul dbe clustered in the same group, then build a model for each cluster
8. how similar train and test customers are???

"""

"\nUnivariate Analysis\n1. When do people order (Distribution of Time of Day) ?\n2. Day of Week (Distribution of day_of_week)?\n3. When do they order again (Distribution of Time Since Prior Order)?\n4. How many prior orders are there (Distribution of Reorders)?\n\n5. how often people order for similar products, say products from the same department\n6. in train data, we have 13000 customers' order history, for every product they bought, then we can use that to predict what other products they reorder in the curr history the latest\n7. while for the test cutomers, they also have history, maybe people who are similar shoul dbe clustered in the same group, then build a model for each cluster\n8. how similar train and test customers are???\n\n"

In [23]:
# department order interval by customers: how often do they buy, mean and std; interval of purchase mean and std
prior_datamart = pd.read_sql_query("SELECT * FROM prior_datamart;", conn)
prior_datamart_2 = prior_datamart.loc[prior_datamart['days_since_prior_order'] != None]
byDepartment = prior_datamart_2.groupby('department')
byDepartment['days_since_prior_order'].describe()

department          
alcohol        count    1.409820e+05
               mean     1.034089e+01
               std      9.024931e+00
               min      0.000000e+00
               25%      4.000000e+00
               50%      7.000000e+00
               75%      1.400000e+01
               max      3.000000e+01
babies         count    4.013150e+05
               mean     1.000447e+01
               std      8.115978e+00
               min      0.000000e+00
               25%      4.000000e+00
               50%      7.000000e+00
               75%      1.300000e+01
               max      3.000000e+01
bakery         count    1.101246e+06
               mean     1.109581e+01
               std      8.750257e+00
               min      0.000000e+00
               25%      5.000000e+00
               50%      8.000000e+00
               75%      1.500000e+01
               max      3.000000e+01
beverages      count    2.522511e+06
               mean     1.105337e+01
               st